# explorer

> CLI tool for API exploration of cjm-fasthtml-tailwind utilities

In [ ]:
#| default_exp cli.explorer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, discover_utility_modules,
    load_code_from_file, display_items_generic, handle_module_not_found,
    find_usage_in_items, get_view_command, format_usage_examples, simple_item_formatter,
    indented_item_formatter, list_utility_modules
)
from cjm_fasthtml_tailwind.cli.factory_extraction import (
    list_all_factories, list_module_factories, get_factory_by_name
)
from cjm_fasthtml_tailwind.cli.example_discovery import (
    list_all_examples, get_example_by_name, list_module_examples
)
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.search import search_all
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.test_code import execute_test_code
from cjm_fasthtml_tailwind.cli.imports import get_recommended_imports
from cjm_fasthtml_tailwind.cli.pattern_scanner import (
    analyze_and_suggest_input, display_input_analysis, 
    InputType, detect_input_type
)
from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config

from cjm_fasthtml_tailwind.cli.display import (
    display_modules, display_module_factories, display_all_factories, 
    display_module_examples, display_all_examples, display_example_source, 
    display_module_helpers, display_helper_source,
    display_all_helpers, display_factory_info, display_search_results, 
    display_core_utility_source, display_core_utilities, display_imports, display_test_code_result
)

from cjm_fasthtml_tailwind.cli.dynamic_examples import (
    get_example_modules, get_example_factories, get_example_features, get_example_helpers, 
    get_example_core_utils, get_combine_classes_example, get_example_test_code
)

## Parser Setup Functions

Helper functions to set up argument parsers for each command:

In [ ]:
#| export
def add_modules_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'modules' command parser."""
    return subparsers.add_parser('modules', help='List all utility modules')

In [ ]:
#| export
def add_factories_parser(
    subparsers,  # TODO: Add type hint and description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Add the 'factories' command parser."""
    if config is None:
        config = get_active_config()
        
    # Get dynamic examples
    example_factories = get_example_factories(config=config)
    
    description = (
        f'List factory instances that generate {config.display_name} utility classes.\n\n'
        'Factories are the primary way to create CSS classes in this library.'
    )
    
    if example_factories:
        description += f'\nExamples: {example_factories}'
    
    parser = subparsers.add_parser(
        'factories', 
        help='List factories',
        description=description
    )
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show factories for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_factory_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'factory' command parser."""
    parser = subparsers.add_parser('factory', help='Show detailed info for a specific factory')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_factories = get_example_factories()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'name',
        type=str,
        help=f'Factory name (e.g., {example_factories})' if example_factories else 'Factory name'
    )
    return parser

In [ ]:
#| export
def add_examples_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'examples' command parser."""
    parser = subparsers.add_parser('examples', help='Show usage examples')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show examples for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_example_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'example' command parser."""
    parser = subparsers.add_parser('example', help='Show source code for a specific example')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_features = get_example_features()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'feature',
        type=str,
        help=f'Feature name (e.g., {example_features})' if example_features else 'Feature name'
    )
    return parser

In [ ]:
#| export
def add_helpers_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'helpers' command parser."""
    parser = subparsers.add_parser('helpers', help='Show helper functions')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show helpers for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_helper_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'helper' command parser."""
    parser = subparsers.add_parser('helper', help='Show source code for a specific helper')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_helpers = get_example_helpers()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'name',
        type=str,
        help=f'Helper function name (e.g., {example_helpers})' if example_helpers else 'Helper function name'
    )
    return parser

In [ ]:
#| export
def add_search_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'search' command parser."""
    # Get dynamic examples
    example_factories = get_example_factories(limit=2)
    
    description = (
        'Search across all library components to find specific functionality.\n\n'
        'SEARCH CAPABILITIES:\n'
        '- Simple text search with partial word matching (case-insensitive by default)\n'
        '- Searches factory names, documentation, examples, helpers, and modules\n'
        '- Use --include-source to also search within code implementations\n\n'
        'WHAT IT SEARCHES:\n'
        '- Factory names and documentation\n'
        '- Example function names, features, and docstrings\n'
        '- Helper function names and docstrings\n'
        '- Module names and documentation\n'
        '- Source code of examples/helpers (with --include-source flag)\n\n'
        'SEARCH BEHAVIOR:\n'
        '- Partial matching\n'
        '- No regex/wildcards\n'
        '- Not all possible CSS class values (values are dynamic, but may be able to find specific values with --include-source flag).\n\n'
        'BEST FOR:\n'
        '- Finding utility categories\n'
        '- Locating factory names\n'
        '- Discovering programming patterns\n\n'
        'TIPS FOR CODING ASSISTANTS:\n'
        '- Use partial words to find related utilities\n'
        '- Add --include-source to find usage patterns in code\n'
        '- Use --in to narrow search scope (factories, examples, helpers, modules)\n'
        '- Results show context and direct commands to view details'
    )
    
    if example_factories:
        description += f'\n\nExample searches: {example_factories}'
    
    parser = subparsers.add_parser(
        'search', 
        help='Search across all library components',
        description=description,
        formatter_class=argparse.RawDescriptionHelpFormatter  # Add this to preserve formatting
    )
    parser.add_argument(
        'query',
        type=str,
        help='Search query (supports partial word matching, case-insensitive by default)'
    )
    parser.add_argument(
        '--in',
        dest='search_in',
        type=str,
        choices=['all', 'factories', 'examples', 'helpers', 'modules'],
        default='all',
        help='What to search in (default: all)'
    )
    parser.add_argument(
        '--include-source',
        action='store_true',
        help='Also search in source code of examples and helpers'
    )
    parser.add_argument(
        '--case-sensitive',
        action='store_true',
        help='Perform case-sensitive search'
    )
    return parser

In [ ]:
#| export
def add_test_code_parser(
    subparsers,  # TODO: Add type hint and description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Add the 'test-code' command parser."""
    if config is None:
        config = get_active_config()
        
    # Get dynamic examples
    example_code = get_example_test_code()
    combine_example = get_combine_classes_example()
    
    description = (
        '⚠️  CRITICAL: Test code snippets using the library with automatic imports.\n\n'
        'THIS COMMAND IS MANDATORY BEFORE IMPLEMENTATION:\n'
        '- ALWAYS validate generated code with this command first\n'
        '- NEVER skip this step - it prevents errors and saves debugging time\n'
        '- This ensures correct syntax, proper imports, and expected output\n\n'
        'This command:\n'
        f'- Automatically generates all necessary {config.package_name} import statements\n'
        '- Executes your code in a safe environment\n'
        '- Shows output or error messages\n'
        '- Validates your understanding before implementation\n\n'
        'Code can be provided directly or loaded from a file.\n\n'
        'Code format notes:\n'
        f'- Single-line statements work: "{example_code}; {example_code}"\n'
        '- Control flow requires multiline format (no semicolons):\n'
        '  Use: $\'for i in [1,2]:\\n    print(p(i))\'\n'
        '  NOT: "for i in [1,2]: print(p(i))"\n'
        '- F-strings are fully supported'
    )
    
    epilog = '⚠️  REMEMBER: Always test code before using it in production!\n\n'
    epilog += 'Examples:\n'
    epilog += f'  {config.cli_command} test-code "{example_code}"\n'
    epilog += f'  {config.cli_command} test-code --file snippet.py\n'
    epilog += f'  {config.cli_command} test-code "{combine_example}" --show-imports'
    
    parser = subparsers.add_parser(
        'test-code', 
        help='⚠️  CRITICAL: Test code snippets using the library (ALWAYS use before implementation)',
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    parser.add_argument(
        'code',
        type=str,
        nargs='?',
        help='Code to test (or use --file to load from file)'
    )
    parser.add_argument(
        '--file', '-f',
        type=str,
        help='Load code from a file'
    )
    parser.add_argument(
        '--show-imports',
        action='store_true',
        help='Show the generated imports'
    )
    parser.add_argument(
        '--timeout',
        type=int,
        default=10,
        help='Execution timeout in seconds (default: 10)'
    )
    return parser

In [ ]:
#| export
def add_core_utils_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'core-utils' command parser."""
    return subparsers.add_parser('core-utils', help='List core utility functions')

In [ ]:
#| export
def add_core_util_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'core-util' command parser."""
    parser = subparsers.add_parser('core-util', help='Show source code for a core utility')
    
    # Get examples dynamically
    example_utils = get_example_core_utils()
    
    parser.add_argument(
        'name',
        type=str,
        help=f'Utility name (e.g., {example_utils})' if example_utils else 'Utility name'
    )
    return parser

In [ ]:
#| export
def add_imports_parser(
    subparsers,  # TODO: Add type hint and description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Add the 'imports' command parser."""
    if config is None:
        config = get_active_config()
    
    # Get dynamic examples
    example_modules = get_example_modules(limit=2)
    
    description = (
        f'Generate recommended {config.package_name} import statements for using the library.\n\n'
        'This command generates:\n'
        '- FastHTML common imports (Div, P, Button, etc.)\n'
        f'- Core utility imports ({[util[0] for util in config.core_utilities]})\n'
        '- Factory imports from selected modules\n'
        '- Helper function imports from selected modules\n\n'
        'By default, includes imports from all available modules.\n'
        'Use --modules to limit imports to specific modules only.'
    )
    
    if example_modules:
        description += f'\n\nExample: --modules {example_modules}'
    
    parser = subparsers.add_parser(
        'imports', 
        help='Show recommended import statements',
        description=description
    )
    parser.add_argument(
        '--modules', '-m',
        type=str,
        nargs='+',
        help='Specific modules to include (default: all modules)'
    )
    return parser

## Command Dispatcher

Function to handle command execution:

In [ ]:
#| export
def add_scan_parser(
    subparsers,  # TODO: Add type hint and description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Add the 'scan' command parser."""
    if config is None:
        config = get_active_config()
        
    description = (
        'Scan Python code for replaceable CSS class patterns.\n\n'
        'This command analyzes Python code to find hardcoded CSS classes\n'
        'that could be replaced with library utilities.\n\n'
        'Features:\n'
        '- Identifies cls= patterns in Python code\n'
        '- Extracts CSS classes from strings and combine_classes calls\n'
        '- Matches against available library utilities\n'
        '- Provides migration suggestions with CLI commands\n\n'
        'Supported inputs:\n'
        '- Direct code strings\n'
        '- Python files (.py)\n'
        '- Jupyter notebooks (.ipynb)\n\n'
        'The tool will auto-detect the input type, or you can specify it explicitly.'
    )
    
    epilog = f'''Examples:
  # Scan a code snippet
  {config.cli_command} scan 'Div("Hello", cls="flex items-center px-4")'
  
  # Scan a Python file
  {config.cli_command} scan app.py
  {config.cli_command} scan --file components/navbar.py
  
  # Scan a Jupyter notebook
  {config.cli_command} scan notebook.ipynb
  
  # Specify input type explicitly
  {config.cli_command} scan --type python_file app.py
  {config.cli_command} scan --type notebook example.ipynb
  
  # Only show analysis without suggestions
  {config.cli_command} scan app.py --no-suggestions'''
    
    parser = subparsers.add_parser(
        'scan', 
        help='Scan code for replaceable CSS patterns',
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    
    parser.add_argument(
        'input',
        type=str,
        help='Code string, Python file path, or Jupyter notebook path'
    )
    
    parser.add_argument(
        '--file', '-f',
        action='store_true',
        help='Treat input as a file path (auto-detected by default)'
    )
    
    parser.add_argument(
        '--type', '-t',
        type=str,
        choices=['code', 'python_file', 'notebook'],
        help='Explicitly specify input type (default: auto-detect)'
    )
    
    parser.add_argument(
        '--no-suggestions',
        action='store_true',
        help='Only show analysis without migration suggestions'
    )
    
    return parser

In [ ]:
#| export
def dispatch_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Dispatch the parsed arguments to the appropriate handler."""
    if args.command == 'modules':
        display_modules()
    
    elif args.command == 'factories':
        if args.module:
            display_module_factories(args.module)
        else:
            display_all_factories()
    
    elif args.command == 'factory':
        display_factory_info(args.module, args.name)
    
    elif args.command == 'examples':
        if args.module:
            display_module_examples(args.module)
        else:
            display_all_examples()
    
    elif args.command == 'example':
        display_example_source(args.module, args.feature)
    
    elif args.command == 'helpers':
        if args.module:
            display_module_helpers(args.module)
        else:
            display_all_helpers()
    
    elif args.command == 'helper':
        display_helper_source(args.module, args.name)
    
    elif args.command == 'search':
        handle_search_command(args)
    
    elif args.command == 'test-code':
        handle_test_code_command(args)
    
    elif args.command == 'core-utils':
        display_core_utilities()
    
    elif args.command == 'core-util':
        display_core_utility_source(args.name)
    
    elif args.command == 'imports':
        display_imports(args.modules)
    
    elif args.command == 'scan':
        handle_scan_command(args)

In [ ]:
#| export
def handle_search_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Handle the search command."""
    # Determine which content types to search
    if args.search_in == 'all':
        content_types = None  # Will search all types
    else:
        content_types = [args.search_in]
    
    # Perform search
    results = search_all(
        args.query,
        content_types=content_types,
        include_source=args.include_source,
        case_sensitive=args.case_sensitive
    )
    
    # Display results
    display_search_results(results, args.query)

In [ ]:
#| export
def handle_test_code_command(
    args,  # TODO: Add type hint and description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Handle the test-code command."""
    if config is None:
        config = get_active_config()
        
    # Get code from command line or file
    code = None
    if args.file:
        code = load_code_from_file(args.file)
        if code is None:
            return  # Error already printed
    elif args.code:
        code = args.code
    else:
        example_test_code = get_example_test_code()
        print("Error: You must provide code to test either as an argument or via --file")
        print("\nExamples:")
        print(f'  {config.cli_command} test-code "{example_test_code}"')
        print(f'  {config.cli_command} test-code --file test_snippet.py')
        return
    
    # Execute the code
    success, stdout, stderr = execute_test_code(
        code,
        show_imports=args.show_imports,
        timeout=args.timeout
    )
    
    # Display results
    display_test_code_result(success, stdout, stderr, code)

## Main Parser Setup

Function to set up the main argument parser:

In [ ]:
#| export
def handle_scan_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Handle the scan command."""
    # Determine input type
    input_type = None
    if args.type:
        # Map string to InputType enum
        type_map = {
            'code': InputType.CODE,
            'python_file': InputType.PYTHON_FILE,
            'notebook': InputType.NOTEBOOK
        }
        input_type = type_map.get(args.type)
    elif args.file:
        # Force file interpretation
        input_type = None  # Let auto-detect handle it
    
    # Display analysis with or without suggestions
    if args.no_suggestions:
        display_input_analysis(args.input, input_type)
    else:
        analyze_and_suggest_input(args.input, input_type)

In [ ]:
#| export
def create_help_description(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
):
    """Create a comprehensive description for the CLI tool's help message"""
    if config is None:
        config = get_active_config()
    
    example_factories = get_example_factories(limit=4, config=config)
    
    return f"""
{config.package_name} CLI Explorer

This tool helps you explore the {config.package_name} library, which provides:
- Python-native {config.display_name} utility class builders for FastHTML projects
- Type-safe, dynamic CSS class generation without hardcoded strings
- Comprehensive utility factories ({example_factories}, etc.)
- Helper functions for common patterns
- Full integration with FastHTML components

Purpose: This CLI tool enables autonomous exploration of the library's API by:
- Discovering all available utility modules and their documentation
- Listing factory instances with their built-in documentation
- Showing usage examples from test functions
- Providing source code for helper functions
- Searching across all library components
- CRITICALLY: Testing code snippets with automatic imports BEFORE implementation
- Generating recommended import statements
- Scanning existing code for replaceable CSS patterns

All information is dynamically extracted from the library itself - nothing is hardcoded.
""".strip()

In [ ]:
#| export
def create_help_epilogue(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
):
    """Create a comprehensive epilogue for the CLI tool's help message"""
    if config is None:
        config = get_active_config()

    # Get dynamic examples
    example_modules = get_example_modules(limit=3, config=config)
    example_factories = get_example_factories(limit=4, config=config)
    first_module = get_example_modules(limit=1, config=config)
    first_factory = get_example_factories(limit=1, config=config)
    example_feature = get_example_features(limit=1, config=config)
    example_test_code = get_example_test_code()
    
    # Build example usage flow with aligned comments
    usage_flow_lines = [
        (f"{config.cli_command} modules", "See what's available"),
        (f"{config.cli_command} factories -m {first_module}", f"Explore {first_module} utilities"),
        (f"{config.cli_command} factory {first_module} {first_factory}", f"Learn about {first_factory} factory"),
        (f"{config.cli_command} example {first_module} {example_feature}", "See usage examples"),
        (f"{config.cli_command} test-code '{example_test_code}'", "CRITICAL: Test your understanding"),
        (f"{config.cli_command} scan app.py", "Analyze existing code"),
    ]
    
    # Calculate max width for alignment
    max_cmd_width = max(len(cmd) for cmd, _ in usage_flow_lines)
    
    # Format lines with aligned comments
    formatted_usage_flow = "\n".join(
        f"  {cmd:<{max_cmd_width}}  # {comment}"
        for cmd, comment in usage_flow_lines
    )
    
    # Create epilog with guidance
    return f"""
Getting Started:
  1. List all modules:     {config.cli_command} modules
  2. View factories:       {config.cli_command} factories
  3. Search for patterns:  {config.cli_command} search <query>
  4. CRITICAL: Test code:  {config.cli_command} test-code "<code>"
  5. Get imports:          {config.cli_command} imports
  6. Scan existing code:   {config.cli_command} scan <file>

Exploration Workflow:
  - Start with 'modules' to see available utility categories
  - Use 'factories -m <module>' to explore specific modules
  - Use 'factory <module> <name>' for detailed factory information
  - Use 'examples' to see test-based usage patterns
  - Use 'search' to find specific functionality
  - CRITICAL: ALWAYS use 'test-code' to validate code BEFORE implementation
  - Use 'scan' to analyze existing code for migration opportunities

IMPORTANT: Code Validation
  ⚠️  ALWAYS use 'test-code' to verify generated code before using it
  ⚠️  This ensures correct syntax, proper imports, and expected output
  ⚠️  Never skip this step - it prevents errors and saves debugging time

Key Concepts:
  - Factories: Objects that generate CSS classes (e.g., {example_factories})
  - Modules: Categories of utilities ({example_modules}, etc.)
  - Examples: Test functions demonstrating usage patterns
  - Helpers: Convenience functions for common patterns

Tips for Coding Assistants:
  - MANDATORY: Use 'test-code' to validate ALL generated code before implementation
  - Use 'search --include-source' to find usage patterns in code
  - Use 'imports' to get all necessary import statements
  - Use 'scan' to identify replaceable hardcoded CSS classes
  - Factory names are intuitive: {example_factories}
  - Combine utilities with combine_classes() function
  - All factories support method chaining and attribute access

Example Usage Flow:
{formatted_usage_flow}
""".strip()

In [ ]:
#| export
def setup_argument_parser(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Set up the main argument parser with all subcommands."""
    if config is None:
        config = get_active_config()

    description = create_help_description()
    epilog = create_help_epilogue()
    
    parser = argparse.ArgumentParser(
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter,
        prog=config.cli_command
    )
    
    # Create subcommands
    subparsers = parser.add_subparsers(dest='command', help='Available commands')
    
    # Add all subcommand parsers
    add_modules_parser(subparsers)
    add_factories_parser(subparsers)
    add_factory_parser(subparsers)
    add_examples_parser(subparsers)
    add_example_parser(subparsers)
    add_helpers_parser(subparsers)
    add_helper_parser(subparsers)
    add_search_parser(subparsers)
    add_test_code_parser(subparsers)
    add_core_utils_parser(subparsers)
    add_core_util_parser(subparsers)
    add_imports_parser(subparsers)
    add_scan_parser(subparsers)  # Add the scan parser
    
    return parser

## CLI Entry Point

Main function for the console script:

In [ ]:
#| export
import argparse

def main(
): # TODO: Add type hint
    """CLI entry point for exploring cjm-fasthtml-tailwind utilities."""
    # Set up the argument parser
    parser = setup_argument_parser()
    
    # Parse arguments
    args = parser.parse_args()
    
    # If no command specified, show help
    if args.command is None:
        parser.print_help()
        return
    
    # Dispatch to the appropriate command handler
    dispatch_command(args)

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()